In [ ]:
#import packages
import pandas as pd
import numpy as np
import random
from random import randint

import urllib
import urllib.request
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import chromedriver_binary
import requests

from IPython.core.display import clear_output

import os

In [ ]:
###THIS CODE WORKS
driver = webdriver.Chrome(executable_path=r"/Users/kelseyheng/Desktop/Metis/projectluther/chromedriver")
athletedatasaved=''
website_list = []
start_time = time.time()
requests = 0

'''loop to get pages'''
pages = [str(i) for i in range(1,2)]
for page in pages:
    url = ('https://games.crossfit.com/leaderboard/open/2019?country_champions=0&division=1&sort=0&scaled=0&page=' + page)
    
    driver.get(url)
    time.sleep(3+2*random.random())
    soup=BeautifulSoup(driver.page_source,'lxml')
    website_list.append(soup.text)
    
    #monitor the request
    requests += 1
    elapsed_time = time.time() - start_time
    print('Request:{}; Frequency:{} requests/s'.format(requests, requests/elapsed_time))
    clear_output(wait=True)
    
    # Break the loop if the number of requests is greater than expected
    if requests > 1000:
        warn('Number of requests was greater than expected.')
        break
        
    '''retrieve name'''
    for record in soup.find('tbody').findAll('tr'):   
        athletedata = record.find_all("div", attrs={"class":"cell-inner"})[0].text
        athletedata = athletedata + ',' + record.find_all("div", attrs={"class": "first-name"})[0].text
        athletedata = athletedata + ',' + record.find_all("div", attrs={"class": "last-name"})[0].text
        
        '''retrieve age, height and weight'''
        for athletestats in record.find_all("ul", attrs={"class": "info"})[0].find_all('li'):
            stat = athletestats.text.replace('Age ','').replace(',','')
            
            if not stat:
                '''height or weight 'li' is empty'''
                stat = ','
               
            elif '|' in stat:
                '''split height and weight using the line'''
                heightweight = stat.split("|")
                heightmetric = heightweight[0].strip().split(' ')
                if 'in' in heightmetric[1]:
                    heightmetric[0] = float(heightmetric[0]) * 2.54
                weightmetric = heightweight[1].strip().split(' ')
                if 'lb' in weightmetric[1]:
                    weightmetric[0] = float(weightmetric[0]) * 0.453592
                stat = str(heightmetric[0]) + ',' + str(weightmetric[0])
                
            elif ' in' in stat or ' cm' in stat:
                print(stat)
                heightmetric = stat.strip().split(' ')
                if 'in' in heightmetric[1]:
                    heightmetric[0] = float(heightmetric[0]) * 2.54
                stat = str(heightmetric[0]) + ','
                
            elif ' lb' in stat or ' kg' in stat:
                weightmetric = stat.strip().split(' ')
                if 'lb' in weightmetric[1]:
                    weightmetric[0] = float(weightmetric[0]) * 0.453592
                stat = ',' + str(weightmetric[0])
          
            athletedata = athletedata + ',' + stat
        
        '''get total points, only one class in td, therefore loop is not needed'''
        totalpoints = record.find_all('td', attrs={'class': 'total-points'})[0].find('div').text
        
        '''retrieve rank and result across all workouts, score is the common class in all workouts'''
        athletedata = athletedata + ',' + totalpoints
        score = ''
        for points in record.find_all('td', attrs={'class': 'score'}):
            rank = points.find_all('span', attrs={'class': 'rank'})[0].text.strip()[:-2]
            result = points.find_all('span', attrs={'class': 'result'})[0].text
            result = result.strip().replace(')','').replace('(','')
            score = score + rank +','+ result + ','
        athletedata = athletedata + ',' + score[:-1]

        #print(athletedata)
        athletedatasaved = athletedatasaved + '\n' + athletedata
        

    header="Rank,First Name,Last Name,Country,Age,Height(cm),Weight(kg),Points,19.1 Rank,19.1 Result,19.2 Rank,19.2 Result,19.3 Rank,19.3 Result,19.4 Rank,19.4 Result,19.5 Rank,19.5 Result" + '\n'
    file = open(os.path.expanduser("crossfitleaderboard_pg1.csv"),'wb')
    file.write(bytes(header, encoding='ascii', errors='ignore'))
    file.write(bytes(athletedatasaved, encoding='ascii', errors='ignore'))

In [2]:
#combine CSV files
#from glob import glob
#filenames = glob('crossfit*.csv')

#df = pd.concat([pd.read_csv(f) for f in filenames])
#df.to_csv('maleleadershipboard_1-600.csv', index =False)

#import combined CSV file
male_df = pd.read_csv('maleleadershipboard_1-600.csv')
male_df.columns = ['rank','first_name','last_name','country','age','height_cm','weight_kg','total_points','191_rank','19.1_result','192_rank','19.2_result','193_rank','19.3_result','194_rank','19.4_result','195_rank','19.5_result']

#drop columns that are not required
male_df.drop(columns=['first_name','last_name','19.1_result','19.2_result','19.3_result','19.4_result','19.5_result'], axis=1, inplace=True)
male_df.head()

In [3]:
# fill NaN with median
# round float off to zero decimal
male_df['height_cm'].fillna(male_df['height_cm'].median(), inplace=True)
male_df['height_cm'] = male_df['height_cm'].round(0)
male_df['weight_kg'].fillna(male_df['weight_kg'].median(), inplace=True)
male_df['weight_kg'] = male_df['weight_kg'].round(0)

#add BMI column
height_m = male_df['height_cm'].div(100)
male_df['bmi'] = male_df['weight_kg'].div(np.square(height_m)).round(1)

male_df = male_df[['rank','country','age','height_cm','weight_kg','bmi','total_points','191_rank','192_rank','193_rank','194_rank','195_rank']]

male_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 12 columns):
rank            30000 non-null int64
country         30000 non-null object
age             30000 non-null int64
height_cm       30000 non-null float64
weight_kg       30000 non-null float64
bmi             30000 non-null float64
total_points    30000 non-null int64
191_rank        30000 non-null int64
192_rank        30000 non-null int64
193_rank        30000 non-null int64
194_rank        30000 non-null int64
195_rank        30000 non-null int64
dtypes: float64(3), int64(8), object(1)
memory usage: 2.7+ MB


In [4]:
#remove inaccurate data by using outliers
out1 = male_df['height_cm'].quantile(0.01)
out2 = male_df['height_cm'].quantile(0.99)
out3 = male_df['weight_kg'].quantile(0.01)
out4 = male_df['weight_kg'].quantile(0.99)

male_df = male_df[male_df['height_cm'] < out2]
male_df = male_df[male_df['height_cm'] > out1]
male_df = male_df[male_df['weight_kg'] < out4]
male_df = male_df[male_df['weight_kg'] > out3]

out5 = male_df['191_rank'].quantile(0.99)
male_df = male_df[male_df['191_rank'] < out5]
out6 = male_df['192_rank'].quantile(0.99)
male_df = male_df[male_df['192_rank'] < out6]
out7 = male_df['193_rank'].quantile(0.99)
male_df = male_df[male_df['193_rank'] < out7]
out8 = male_df['194_rank'].quantile(0.99)
male_df = male_df[male_df['194_rank'] < out8]
out9 = male_df['195_rank'].quantile(0.99)
male_df = male_df[male_df['195_rank'] < out9]

male_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27096 entries, 0 to 29999
Data columns (total 12 columns):
rank            27096 non-null int64
country         27096 non-null object
age             27096 non-null int64
height_cm       27096 non-null float64
weight_kg       27096 non-null float64
bmi             27096 non-null float64
total_points    27096 non-null int64
191_rank        27096 non-null int64
192_rank        27096 non-null int64
193_rank        27096 non-null int64
194_rank        27096 non-null int64
195_rank        27096 non-null int64
dtypes: float64(3), int64(8), object(1)
memory usage: 2.7+ MB


In [5]:
male_df.to_csv('maleleadershipboard_ranksout.csv', index =False)
